In [1]:
##%matplotlib widget
## with %matplotlib notebook: seems to require ipympl as part of environment, either
## part of the conda environment or "pip install ipympl"
## otherwise, does not show ANY plots in notebook, plt.savefig() works
%matplotlib notebook  
##%matplotlib inline    ## --plt.savefig()  works, but re-sizing does NOT

This notebook is a short demo to illustrate execution.   For odd historical reasons, it uses "toy Monte Carlo" (simulated data)for "training" and "full LHCB MC" for validation.

The network architecture is a FourFeature model that has the same basic structure as the TwoFeature 6 convolutional layer  model. The differences/similarlities are:

  [1]  the FourFeature model has four channels of input rather than two.
  [2]  the extra two channels are added by using load_xy=True in collect_data (x2)
  [3]  the pretrained_dict is used with strict=False
  
In the first iterations, the weights from a trained TwoFeature_CNN6Layer_A model should be reused for the (X,Xsq) part of the algorithm, and the algorithm should only learn the filter for the (perturbative) (x,y) features.
  
Once the perturbative filters are beng generated, the algorithm can start with weights from a previous iteration of this algorithm and all weights free to float.

Check the current GPU usage. Please try to be nice!

In [2]:
!nvidia-smi

Thu Jun 11 10:13:03 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN V             Off  | 00000000:03:00.0 Off |                  N/A |
| 41%   59C    P2   114W / 250W |   3714MiB / 12066MiB |     90%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:83:00.0 Off |                    0 |
| N/A   35C    P0    34W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

> **WARNING**: The card numbers here are *not* the same as in CUDA. You have been warned.

## Imports

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import time
import torch
import pandas as pd
import mlflow

# Python 3 standard library
from pathlib import Path

from torchsummary import summary

### Set up local parameters

In [4]:
class Params(object):
    def __init__(self, batch_size, epochs, lr):
        self.batch_size = batch_size
        self.epochs = epochs
        self.lr = lr
        
args = Params(128, 10, 0.000025)

Make the output directory if it does not exist:

## Get the helper functions

Add the directory with the model
definitions to the path so we can import from it:

> When you type `import X`,
Python searches `sys.path` for a python
file named `X.py` to import. So we need to add the model directory to the path.

In [5]:
# From model/collectdata.py
from model.collectdata_mdsA import collect_data

# From model/loss.py
##from loss import Loss
from model.alt_loss_A import Loss

# From model/training.py
from model.training import trainNet, select_gpu

# From model/models.py
##  will start with model from TwoFeatures_CNN6Layer_A in the first instance
##  see relevant cell below


from model.models_mds_01June20 import SimpleCNN5Layer_Ca as Model

from model.training import trainNet, select_gpu, Results
from model.plots import dual_train_plots, replace_in_ax

from mlflow import pytorch

In [6]:
# This gets built up during the run - do not rerun this cell
results = pd.DataFrame([], columns=Results._fields)

Set up Torch device configuration. All tensors and model parameters need to know where to be put.
This takes a BUS ID number: The BUS ID is the same as the listing at the top of this script.

In [7]:
device = select_gpu(1)

1 available GPUs (initially using device 0):
  0 Tesla P100-PCIE-16GB


## Loading data

Load the dataset, split into parts, then move to device (see `collectdata.py` in the `../model` directory)

In [8]:
## newer vernacular
## Training dataset. You can put as many files here as desired.

## in this DEMO example we use only one 80K training set -- the model starts with well-trained weights,
## and using a smaller training set reduces both the time to load the data and the time to train an epoch
##  set the option load_XandXsq = True to use both DKE and KDE^2 as input features
train_loader = collect_data('/share/lazy/sokoloff/ML-data_AA/Aug14_80K_train.h5',
                            '/share/lazy/sokoloff/ML-data_AA/Oct03_80K_train.h5',
##                            'dataAA/Oct03_80K2_train.h5',
                             batch_size=args.batch_size,
## if we are using a larger dataset (240K events, with the datasets above, and 11 GB  of GPU memory),
## not the dataset will overflow the GPU memory; device=device will allow the data to move back
## and forth between the CPU and GPU memory. While this allows use of a larger dataset, it slows
## down performance by about 10%.  So comment out when not needed.
##                           device=device,
                            masking=True, shuffle=True,
                            load_XandXsq=False,
                            load_xy=False)

# Validation dataset. You can slice to reduce the size.
## dataAA -> /share/lazy/sokoloff/ML-data_AA/
val_loader = collect_data('/share/lazy/sokoloff/ML-data_AA/Oct03_20K_val.h5',
## mds val_loader = collect_data('dataAA/HLT1CPU_1kevts_val.h5',

                          batch_size=args.batch_size,
                          slice=slice(256 * 39),
                          device=device,
                          masking=True, shuffle=False,
                          load_XandXsq=False,
                          load_xy=False)

Loading data...
Loaded /share/lazy/sokoloff/ML-data_AA/Aug14_80K_train.h5 in 16.94 s
Loaded /share/lazy/sokoloff/ML-data_AA/Oct03_80K_train.h5 in 15.36 s
Constructing 160000 event dataset took 0.3614 s
Loading data...
Loaded /share/lazy/sokoloff/ML-data_AA/Oct03_20K_val.h5 in 3.499 s
Constructing 9984 event dataset took 2.849 s


KeyboardInterrupt: 

# Preparing the model

Prepare a model, use multiple GPUs if they are VISIBLE, and move the model to the device.

In [ ]:
model = Model()
##  mds 200121 loss = Loss(epsilon=1e-5,coefficient=1.0)
loss = Loss(epsilon=1e-5,coefficient=2.5)

In [ ]:
print("Let's use", torch.cuda.device_count(), "GPUs!")
if torch.cuda.device_count() > 1:
    model = torch.nn.DataParallel(model)

Let's move the model's weight matricies to the GPU:

print('output = ',output)
##print('oldOutput = ',oldOutput)
##  use the first four layers from a pre-existing model
##  see example at https://discuss.pytorch.org/t/how-to-load-part-of-pre-trained-model/1113

##   ML -> /share/lazy/sokoloff/ML
pretrained_dict = torch.load('ML/Aug17_FourFeature_CNN6LayerPlus_TargetsAA_Loss_A_1p0_final.pyt')
model_dict = model.state_dict()
## mds 190725 for debugging
print("for model_dict")
index = 0
for k,v in model_dict.items():
    print("index, k =  ",index,"  ",k)
    index = index+1
    
print(" \n","  for pretrained_dict")
index = 0
for k,v in pretrained_dict.items():
    print("index, k =  ",index,"  ",k)
    index = index+1
## mds  

print("model_dict instantiated")
# 1. filter out unnecessary keys
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
## print("pretrained_dict iterated")
# 2. overwrite entries in the existing state dict
model_dict.update(pretrained_dict) 
##
#   when starting from a model with a fully connected last layer rather than a convolutional layer
# 3. load the new state dict
#   need to use strict=False as the two models state model attributes do not agree exactly
#   see https://pytorch.org/docs/master/_modules/torch/nn/modules/module.html#Module.load_state_dict

model.load_state_dict(pretrained_dict,strict=False)

## print('model_dict =    ', model_dict)


In [ ]:
model = model.to(device)

## Create Experiment

The first line sets the location to save the experiment. This will usually appear as a "0", "1", and so on (depending on how many experiments you have already created in that directory) in the specified folder. The next line sets the experiment name, which will be visible when launching MLFlow through localhost. This will be a piece of metadata and does not create a directory or file.

In [ ]:
mlflow.tracking.set_tracking_uri('file:/share/lazy/pv-finder_model_repo')
mlflow.set_experiment('SimpleCNN5Layer_Ca')

## Train 

The first for loop saves the model and its corresponding data to the experiment so that it may be accessed in MLFlow.

The body of this loop runs once per epoch. Results is a named tuple of values (loss per epoch for training and validation, time each). Log this data into the mlflow experiment set in the previous cell.

In [ ]:

model = Model().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

we_are_training_from_scratch = True

if not we_are_training_from_scratch:
    # If we are not training from scratch, this path should be the path to the "run_stats" file in the artifacts 
    # directory of whatever run you are using as a baseline. 
    # You can find the path in the MLFlow UI. It should end in /artifacts/run_stats
    PATH = '/share/lazy/pv-finder_model_repo'
    
    # Load the model and optimizer state_dict, and the total number of epochs
    # The use case for this is if we care about the optimizer state_dict, which we do if we have multiple training 
    # sessions with momentum and/or learning rate decay. this will track the decay/momentum.
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint['model'])
    epoch_start = checkpoint['epoch']
    
    # do this so it does not use the learning rate from the previous run. this is unwanted behavior
    # in our scenario since we are not using a learning rate scheduler, rather we want to tune the learning
    # rate further after we have gotten past the stalling
    checkpoint['optimizer']['param_groups'][0]['lr'] = args.lr
    optimizer.load_state_dict(checkpoint['optimizer'])

else:
    epoch_start = 0

run_name = 'Run 3'

with mlflow.start_run(run_name = run_name) as run:
# with mlflow.start_run() as run:

    for key, value in vars(args).items():
        mlflow.log_param(key, value)
        mlflow.set_tag('Optimizer', 'Adam')
        
    for result in trainNet(model, optimizer, loss,
                            train_loader, val_loader,
                            args.epochs+epoch_start, epoch_start=epoch_start,
                            notebook=True):

        result = result._asdict()
        mlflow.log_metric('Efficiency', result['eff_val'].eff_rate, result['epoch'])
        mlflow.log_metric('False Positive Rate',  result['eff_val'].fp_rate, result['epoch'])
        mlflow.log_metric('Validation Loss',  float(result['cost']), int(result['epoch']))
        
        if result['eff_val'].eff_rate == 0:
            mlflow.set_tag('Stalled', 'True')
        else:
            mlflow.set_tag('Stalled', 'False')

    # Save model AND optimizer state_dict AND epoch number.
    torch.save({
        'model':model.state_dict(),
        'optimizer':optimizer.state_dict(),
        'epoch':args.epochs+result['epoch']
        }, 'run_stats.pyt')
    mlflow.log_artifact('run_stats.pyt')

In [ ]:
##quit()